# WARNING: debugged, yet to tune-up

### Possible improvements:
1) check "choose_prob" method for useless calls, as they rely on the slow np.linalg library

2) consider improving Pr0fromScipy calls (eventually re/move the dimensional check from there)

3) performances for 2+ parameters are poor especially if the true parameters differ a lot. the choice of the probe_state for mixed operators may not be efficient. I have impression that without bootstrapping the model from single operators, the learning process is almost uneffective

...

In [1]:
%matplotlib inline
import qinfer as qi
import qutip as qt
import numpy as np
import scipy as sp
import math as mth
from IPython.display import display, Math, Latex
from functools import partial
import matplotlib.pyplot as plt

In [2]:
import datetime
import os as os
import time as time

In [3]:
import importlib as imp

In [4]:
import logging as logging
import warnings as warnings
import sys

%run LoggingCode.py

Importing Local Function for Hamiltonian learning

In [5]:
# Add personalized library to PYTHONPATH
sys.path.append(os.path.join("..","Libraries","QML_lib"))

In [6]:
from Norms import *
from IOfuncts import *
from EvalLoss import *

import ProbeStates as pros
import multiPGH as mpgh
import GenSimQMD_IQLE as gsi
import Evo as evo

In [7]:
""" filter only the Weight Clipping warning """
#SET LEVEL WARNINGS TO 0 In FINAL VERSION
warnings.filterwarnings("ignore", message='Negative weights occured', category=RuntimeWarning)

# Model learning functions


#ChosenOpList = np.array([evo.sigmaz(), evo.sigmax()])


TrueOpList is the array of the Operators which compose the TRUE Hamiltonian  of the system
TrueParams (array) are the corresponding True values which together with OpList let us write the true Hamiltonian of the system

Hsys= Sum TrueOpList[i].TrueParams[i]


SimParams list of parameters tried for the simulator which will be used as centre of the distribution for the particles
OpList list of the operators tried for the quantum simulator i.e. of the model we are testing

Hsim= Sum OpList[i].SimParams[i]

Particles is the array with all the particles contained in this model, each element of this array is an array
with length equal to the number of operators/Trueparams in the Hamiltonian

Weights are the weights of the particles
    
BayesFactorList is the list with all the BayesFactors of this model
    
VolumeList array containing Volume vs number of experiments
    
NumParticles is the number of particles

    

In [8]:
lst = [evo.sigmax(),evo.sigmay(), evo.sigmaz()]     

In [9]:
import Utils as uti
import Models as mods
import BayesF as bayf
import ModLearn as mdl
import QMD as qmd
import QML as qml

## Reload when needed

In [48]:
imp.reload(pros)
imp.reload(mpgh)
imp.reload(gsi)
imp.reload(evo)
imp.reload(mdl)
imp.reload(uti)
imp.reload(mods)
imp.reload(bayf)
imp.reload(qmd)
imp.reload(qml)

<module 'QML' from '..\\Libraries\\QML_lib\\QML.py'>

In [49]:
# reload(pros)
# reload(mpgh)
# reload(gsi)
# reload(evo)
# reload(mdl)
# reload(uti)
# reload(mods)
# reload(bayf)
# reload(qmd)
# reload(qml)

# MODEL LEARNING TEST

In [62]:
modeltest = qmd.ModelsDevelopmentClass()

print(modeltest.TrueOpList)
print(modeltest.TrueParamsList)
print(uti.ModelNamesPauli([modeltest.TrueOpList], uti.PauliNames() ) )

[[[ 0.+0.j  1.+0.j]
  [ 1.+0.j  0.+0.j]]]
[[ 0.47081445]]
['sx_']


In [63]:
print(modeltest.TrueParamsList)

[[ 0.47081445]]


In [64]:
 for i in range(0, modeltest.MaxModNum):
            print(modeltest.AvailableModsOpList[i])

[[[ 0.+0.j  1.+0.j]
  [ 1.+0.j  0.+0.j]]]
[[[ 0.+0.j -0.-1.j]
  [ 0.+1.j  0.+0.j]]]
[[[ 1.+0.j  0.+0.j]
  [ 0.+0.j -1.+0.j]]]


In [65]:
print('ModelNames: ' + repr(modeltest.ModelNames))
# print('ModelDict: ' + repr(modeltest.ModelDict))

ModelNames: ['sx_', 'sy_', 'sz_', 'sy_sx_', 'sz_sx_', 'sz_sy_', 'sz_sy_sx_']


In [66]:
modeltest.InitialiseAllActiveModels()

[[ 0.85126395]]
Initialization Ready
[[ 0.68291124]]
Initialization Ready
[[ 0.24397817]]
Initialization Ready


In [67]:
start=time.clock()
modeltest.UpdateAllActiveModels(expnum=50)
end=time.clock()
print('Total elapsed time: ' + str(end-start))


Initial time selected > 10.1346215522
Final time selected > 2.94264588982
Final Parameters mean and stdev:[ 0.52185753  0.17739955]
Batch single time 0 elapsed time: 5.1459327354587
Initial time selected > 1.13014864957
Final time selected > 109.911988157
Final Parameters mean and stdev:[-0.12973979  0.03034559]
Batch single time 1 elapsed time: 5.18456105223413
Initial time selected > 7.45999724312
Final time selected > 15.7573692848
Final Parameters mean and stdev:[ 0.07940611  0.02550491]
Batch single time 2 elapsed time: 7.068690523978148
LogTotLikelihoods updated
LogTotLikelihoods updated
LogTotLikelihoods updated
Total elapsed time: 30.493884440747934


In [70]:
modeltest.ComputeAllBayesFactors()

{"['sx_']VS['sy_']": 38517998387403864.0,
 "['sx_']VS['sz_']": 3.4210592069075635e+22,
 "['sy_']VS['sx_']": 0.0,
 "['sy_']VS['sz_']": 888171.59513312532,
 "['sz_']VS['sx_']": 0.0,
 "['sz_']VS['sy_']": 1.125908558052835e-06}

In [71]:
modeltest.ChampionPosition()

[0]

In [72]:
modeltest.ModelNames[1]

'sy_'

In [73]:
modeltest.ChampionPruning()

In [74]:
modeltest.ModelsList

In [19]:
bayesllocal=np.array(modeltest.BayesFactorsList)
# np.max(bayesllocal)
lengthlsit=int(bayesllocal.size)

In [21]:
xsize = int(len(modeltest.ModelsList))
ysize = int(lengthlsit/len(modeltest.ModelsList))

bayessorted=np.reshape(bayesllocal, (xsize,ysize) )
champion = []
champ_threshold=1000

for i in range( ysize ):
    if np.all(bayessorted[i,:] > champ_threshold):
        champion.append(i)
        
    

In [20]:
bayesllocal

array([  4.31817547e+09,   4.60841254e-10,   2.31579311e-10,
         0.00000000e+00,   2.16994451e+09,   9.37020116e+18])

In [23]:
modeltest.BayesFactorsList

[4318175472.0639925,
 4.6084125404632914e-10,
 2.3157931128281461e-10,
 0.0,
 2169944509.6501694,
 9.3702011573112893e+18,
 4318175472.0639925,
 4.6084125404632914e-10,
 2.3157931128281461e-10,
 0.0,
 2169944509.6501694,
 9.3702011573112893e+18]

In [22]:
modeltest.ChampionPosition()

IndexError: index 3 is out of bounds for axis 0 with size 3

In [90]:
i=0
bayf.BayesFactorfromLogL(modeltest.ModelsList[int(outlst[2*i])].KLogTotLikelihood, modeltest.ModelsList[int(outlst[2*i+1])].KLogTotLikelihood)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  from ipykernel import kernelapp as app


5.2586846003288557e-06

In [82]:
modeltest.ModelsList[0].KLogTotLikelihood

array([], dtype=float64)

In [73]:
for i in range(len(modeltest.ModelsList)):
    print(modeltest.ModelsList[i].NewEval)

[ 1.12591301]
[ 0.33413277]
[ 0.05238769]


In [29]:
print(uti.ModelNamesPauli([modeltest.TrueOpList], uti.PauliNames() ) )

['sy_sx_']


In [35]:
modeltest.ComputeAllBayesFactors(fromLogL = False)

{"['sx_']VS['sx_sy_']": 3325025988887.5322,
 "['sx_']VS['sx_sy_sz_']": 0.14551132638824071,
 "['sx_']VS['sx_sz_']": 3.0034087843989771e+25,
 "['sx_']VS['sy_']": 2.0923602628286631e-08,
 "['sx_']VS['sy_sz_']": 2.6497049245066684e-05,
 "['sx_']VS['sz_']": 3.5052248674050073e+27,
 "['sx_sy_']VS['sx_']": 3.0075941737095491e-13,
 "['sx_sy_']VS['sx_sy_sz_']": 4.3742787170231168e-14,
 "['sx_sy_']VS['sx_sz_']": 9032737772386.0742,
 "['sx_sy_']VS['sy_']": 0.0,
 "['sx_sy_']VS['sy_sz_']": 0.0,
 "['sx_sy_']VS['sz_']": 1054194727836628.1,
 "['sx_sy_sz_']VS['sx_']": 6.8723172609387548,
 "['sx_sy_sz_']VS['sx_sy_']": 22850633496501.742,
 "['sx_sy_sz_']VS['sx_sz_']": 2.0640378030680175e+26,
 "['sx_sy_sz_']VS['sy_']": 1.4379363577621262e-07,
 "['sx_sy_sz_']VS['sy_sz_']": 0.00018209612889097393,
 "['sx_sy_sz_']VS['sz_']": 2.4089017359739192e+28,
 "['sx_sz_']VS['sx_']": 0.0,
 "['sx_sz_']VS['sx_sy_']": 1.1068923555512811e-13,
 "['sx_sz_']VS['sx_sy_sz_']": 0.0,
 "['sx_sz_']VS['sy_']": 0.0,
 "['sx_sz_']VS['s

In [36]:
tpool = modeltest.DataPool(20)

In [37]:
modeltest.UpdateAllLogLikelihoods(tpool)

/Users/raf/Anaconda/anaconda/lib/python2.7/site-packages/qinfer/smc.py:285: ApproximationWarning: Extremely small n_ess encountered (5.76431259964). Resampling is likely to fail. Consider adding particles, or resampling more often.
  ApproximationWarning


LogTotLikelihoods updated
LogTotLikelihoods updated
LogTotLikelihoods updated


/Users/raf/Anaconda/anaconda/lib/python2.7/site-packages/qinfer/smc.py:285: ApproximationWarning: Extremely small n_ess encountered (2.60077288946). Resampling is likely to fail. Consider adding particles, or resampling more often.
  ApproximationWarning


LogTotLikelihoods updated


/Users/raf/Anaconda/anaconda/lib/python2.7/site-packages/qinfer/smc.py:285: ApproximationWarning: Extremely small n_ess encountered (1.00314974513). Resampling is likely to fail. Consider adding particles, or resampling more often.
  ApproximationWarning


LogTotLikelihoods updated


/Users/raf/Anaconda/anaconda/lib/python2.7/site-packages/qinfer/smc.py:285: ApproximationWarning: Extremely small n_ess encountered (1.99079302933). Resampling is likely to fail. Consider adding particles, or resampling more often.
  ApproximationWarning


LogTotLikelihoods updated


/Users/raf/Anaconda/anaconda/lib/python2.7/site-packages/qinfer/smc.py:285: ApproximationWarning: Extremely small n_ess encountered (2.68904305164). Resampling is likely to fail. Consider adding particles, or resampling more often.
  ApproximationWarning


LogTotLikelihoods updated


In [39]:
modeltest.ComputeAllBayesFactors(fromLogL = True)

{"['sx_']VS['sx_sy_']": 2.2204460492503131e-15,
 "['sx_']VS['sx_sy_sz_']": 0.0,
 "['sx_']VS['sx_sz_']": 0.14502613261285879,
 "['sx_']VS['sy_']": 0.0,
 "['sx_']VS['sy_sz_']": 0.0,
 "['sx_']VS['sz_']": 2.3079013834153211e+21,
 "['sx_sy_']VS['sx_']": 450645707361296.94,
 "['sx_sy_']VS['sx_sy_sz_']": 1.6082749950685482e-06,
 "['sx_sy_']VS['sx_sz_']": 65355404117195.0,
 "['sx_sy_']VS['sy_']": 0.0,
 "['sx_sy_']VS['sy_sz_']": 0.0,
 "['sx_sy_']VS['sz_']": 1.0400458514493131e+36,
 "['sx_sy_sz_']VS['sx_']": 2.8020438591178857e+20,
 "['sx_sy_sz_']VS['sx_sy_']": 621784.21170922159,
 "['sx_sy_sz_']VS['sx_sz_']": 4.0636958429947707e+19,
 "['sx_sy_sz_']VS['sy_']": 0.0,
 "['sx_sy_sz_']VS['sy_sz_']": 0.0,
 "['sx_sy_sz_']VS['sz_']": 6.4668408988485734e+41,
 "['sx_sz_']VS['sx_']": 6.8953090176475866,
 "['sx_sz_']VS['sx_sy_']": 1.532107773982716e-14,
 "['sx_sz_']VS['sx_sy_sz_']": 0.0,
 "['sx_sz_']VS['sy_']": 0.0,
 "['sx_sz_']VS['sy_sz_']": 0.0,
 "['sx_sz_']VS['sz_']": 1.5913693220905004e+22,
 "['sy_']VS[

# On a Single pair

In [56]:
tpoolpair=modeltest.DataPairPool(5,6, 30)

In [57]:
modeltest.UpdatePairModelLogLikelihoods(5,6, tpoolpair)

LogTotLikelihoods updated


In [58]:
modeltest.ModelsList[5].KLogTotLikelihood

-223.29621241307598

In [60]:
modeltest.ModelsList[6].KLogTotLikelihood

-210.83476612936602

In [63]:
bayf.BayesFactorfromLogL(modeltest.ModelsList[5].KLogTotLikelihood, modeltest.ModelsList[6].KLogTotLikelihood)

3.8731350683329779e-06

In [64]:
bayf.BayesFactorfromLogL(modeltest.ModelsList[6].KLogTotLikelihood, modeltest.ModelsList[5].KLogTotLikelihood)

258188.7753385962

In [33]:
modeltest.ComputeAllBayesFactors(fromLogL = True)

{"['sx_']VS['sx_sy_']": 3.2904721885472413e+84,
 "['sx_']VS['sy_']": 3.4397133939597899e+35,
 "['sx_']VS['sz_']": 2.7678165852015967e+43,
 "['sx_']VS['sz_sx_']": 1.280510537376759e+63,
 "['sx_']VS['sz_sy_']": 1.9707159097095937e+59,
 "['sx_sy_']VS['sx_']": 0.0,
 "['sx_sy_']VS['sy_']": 0.0,
 "['sx_sy_']VS['sz_']": 0.0,
 "['sx_sy_']VS['sz_sx_']": 0.0,
 "['sx_sy_']VS['sz_sy_']": 0.0,
 "['sy_']VS['sx_']": 0.0,
 "['sy_']VS['sx_sy_']": 9.566123137832882e+48,
 "['sy_']VS['sz_']": 80466488.576110482,
 "['sy_']VS['sz_sx_']": 3.7227245142730871e+27,
 "['sy_']VS['sz_sy_']": 5.7293026598384997e+23,
 "['sz_']VS['sx_']": 0.0,
 "['sz_']VS['sx_sy_']": 1.1888331785205981e+41,
 "['sz_']VS['sy_']": 1.2427533668812885e-08,
 "['sz_']VS['sz_sx_']": 4.6264284426328482e+19,
 "['sz_']VS['sz_sy_']": 7120110198942444.0,
 "['sz_sx_']VS['sx_']": 0.0,
 "['sz_sx_']VS['sx_sy_']": 2.5696564709948187e+21,
 "['sz_sx_']VS['sy_']": 0.0,
 "['sz_sx_']VS['sz_']": 0.0,
 "['sz_sx_']VS['sz_sy_']": 0.00015390079598620598,
 "['sz

## Preliminary trials

In [27]:
tpool = modeltest.DataPool(10)

In [33]:
idMod=3

mymodel = modeltest.ModelsList[idMod]

mytpool = np.empty(0)
for j in range( len(tpool) ):
    if idMod is not j:
        mytpool=np.append(mytpool, tpool[j])
print('mytpool len: ', len(mytpool))
mytpool

mytpool len:  60


array([  3.56836995e-01,   1.58083401e+00,   1.09448136e+00,
         1.86424750e+00,   3.57263671e-01,   1.42073129e+00,
         5.29827653e-01,   1.20131291e+00,   8.55761623e-01,
         1.72847692e+00,   2.64867230e+02,   3.19450833e+02,
         1.13337158e+03,   1.54666241e+02,   1.31289569e+03,
         9.10214242e+02,   4.02381052e+02,   8.83574274e+02,
         7.75353685e+03,   2.84264037e+03,   1.71698284e+02,
         6.53238557e+01,   2.73964323e+02,   6.66713766e+02,
         1.76138709e+02,   3.66000634e+02,   1.15482096e+03,
         3.45467819e+03,   3.95268035e+04,   7.40191641e+02,
         7.33844757e+01,   2.16545898e+02,   4.49400050e+01,
         7.63788501e+01,   3.16963256e+02,   3.33724672e+01,
         5.26799673e+01,   5.25487900e+01,   4.91751693e+02,
         8.48642342e+01,   1.73361067e+02,   6.74491520e+02,
         2.16729790e+02,   2.99624902e+02,   7.71997715e+02,
         3.02992719e+02,   2.51454786e+02,   6.19146652e+02,
         8.39861351e+02,

In [34]:
"""This can be adopted alternatively to sample the frequencies"""
#myexperiments = np.array(list(map(lambda x: Kmodel.Heuristic(), range(len(tpool))) ) )

myexperiments = np.empty((len(mytpool), ), dtype=mymodel.GenSimModel.expparams_dtype)       #initialises the experiments to perform for the update

myexperiments['t']  = mytpool

In [35]:
myexperiments

array([(0.35683699543121394, 9.46765942e-315, 6.917e-321),
       (1.5808340135250298, 0.6154953177229143, 0.0),
       (1.0944813590553506, 8.60973318409419e-67, 9.462515493683218e-96),
       (1.864247502033219, 5.937421302973274e-38, 5.981490944369225e-154),
       (0.3572636712171276, 7.128903417486503e-67, 2.2137974609192766e-52),
       (1.4207312855098175, 3.221777067163919e-57, 2.440043148419846e-154),
       (0.5298276533358683, 1.7973950845951164e-52, 1.6584870046017555e-76),
       (1.2013129096038775, 4.0139765785239013e-57, 1.1447266613021176e-259),
       (0.8557616232532389, 8.238801249426261e-67, 8.232977426101218e-67),
       (1.728476919127297, 3.507440597248618e-33, 5.90255903737772e-96),
       (264.86723025964, 4.6569859217993704e-33, 1.0860392870200448e-42),
       (319.45083288907455, 8.230073927580902e-67, 3.856824477342548e-110),
       (1133.3715763774308, 1.0289754681224096e-71, 2.175299784204236e-76),
       (154.66624113126255, 8.230073927580902e-67, 3.8568

In [ ]:
myexperiments[field_i]

In [43]:
inv_field = [item[0] for item in mymodel.GenSimModel.expparams_dtype[1:] ]
print('inv_field: ', inv_field)
for i in range(len(inv_field)):
    myexperiments[inv_field[i]] = mymodel.NewEval[i]
print(myexperiments)

inv_field:  ['w_1', 'w_2']
[(0.35683699543121394, 0.4867189654629474, 0.728015489734225)
 (1.5808340135250298, 0.4867189654629474, 0.728015489734225)
 (1.0944813590553506, 0.4867189654629474, 0.728015489734225)
 (1.864247502033219, 0.4867189654629474, 0.728015489734225)
 (0.3572636712171276, 0.4867189654629474, 0.728015489734225)
 (1.4207312855098175, 0.4867189654629474, 0.728015489734225)
 (0.5298276533358683, 0.4867189654629474, 0.728015489734225)
 (1.2013129096038775, 0.4867189654629474, 0.728015489734225)
 (0.8557616232532389, 0.4867189654629474, 0.728015489734225)
 (1.728476919127297, 0.4867189654629474, 0.728015489734225)
 (264.86723025964, 0.4867189654629474, 0.728015489734225)
 (319.45083288907455, 0.4867189654629474, 0.728015489734225)
 (1133.3715763774308, 0.4867189654629474, 0.728015489734225)
 (154.66624113126255, 0.4867189654629474, 0.728015489734225)
 (1312.8956920887006, 0.4867189654629474, 0.728015489734225)
 (910.2142421090025, 0.4867189654629474, 0.728015489734225)
 (

In [47]:
mysimparams = mymodel.SimParams

mydata = mymodel.GenSimModel.simulate_experiment(mymodel.SimParams, myexperiments)[0][0]

Kupdater=mymodel.Updater
Kupdater.batch_update(mydata, myexperiments, resample_interval=100)

LogLikelihood = np.sum(Kupdater.log_total_likelihood)

# MULTIPLE testing running

In [10]:
nruns = 100
pool_len = 50

collectTrueOpList = []
collectTrueNames = []
collectTrueParams = []

collectEstimParams = []

collectKfactorsDict = []
collectLogL = []


for run in range(nruns):
    modeltest = qmd.ModelsDevelopmentClass(7, lst)
    collectTrueOpList.append(modeltest.TrueOpList)
    collectTrueNames.append(modeltest.TrueNames)
    
    modeltest.InitiliaseAllModels()
    
    collectTrueParams.append(modeltest.TrueParamsList)
    
    start=time.clock()
    modeltest.UpdateAllModels(expnum=350)
    end=time.clock()
    print('Total elapsed time for the learning process: ' + str(end-start))
    
    
    collectEstimParams.append([modeltest.ModelsList[i].NewEval for i in range(len(modeltest.ModelsList)) ])
    
    tpool = modeltest.DataPool(pool_len)
    
    start=time.clock()
    modeltest.UpdateAllLogLikelihoods(tpool)
    end=time.clock()
    print('Total elapsed time for Bayes factor update: ' + str(end-start))
    
    collectKfactorsDict.append(modeltest.ComputeAllBayesFactors(fromLogL = True))
    
    collectLogL.append([modeltest.ModelsList[i].KLogTotLikelihood for i in range(len(modeltest.ModelsList)) ])
    

[[ 0.5860098]]
Initialization Ready
[[ 0.15836811]]
Initialization Ready
[[ 0.4279087]]
Initialization Ready
[[ 0.15672626  0.97625095]]
Initialization Ready
[[ 0.46922643  0.24787462]]
Initialization Ready
[[ 0.6439683   0.20564304]]
Initialization Ready
[[ 0.63293011  0.85017443  0.10804446]]
Initialization Ready
Initial time selected > 1.29106627735
Final time selected > 1.40759000391
Final Parameters mean and stdev:[-0.01985354  0.51262223]
True model was: [[[ 0.+0.j  1.+0.j]
  [ 1.+0.j  0.+0.j]]

 [[ 0.+0.j -0.-1.j]
  [ 0.+1.j  0.+0.j]]

 [[ 1.+0.j  0.+0.j]
  [ 0.+0.j -1.+0.j]]]
True parameters were: [[ 0.43405429  0.77906936  0.83532925]]
Single iteration 0 elapsed time: 52.20132462257923
Initial time selected > 6.72291585205
Final time selected > 1284.91366825
Exiting learning for Reaching Cov. Norm. Thrshold of 4.24891691164e-07
 at Iteration Number 188
Final Parameters mean and stdev:[  1.67007387e+00   6.51837166e-04]
True model was: [[[ 0.+0.j  1.+0.j]
  [ 1.+0.j  0.+0.j]]



KeyboardInterrupt: 

In [25]:
modeltest = qmd.ModelsDevelopmentClass(7, lst)
modeltest.TrueParamsList

array([[ 0.50250749]])

## Data export

In [ ]:
import pickle
PIK = "testQMD_pool"+str(pool_len)+".dat"

In [514]:
data = [collectTrueOpList, collectTrueNames, collectTrueParams, collectEstimParams, collectKfactorsDict, collectLogL]
with open(PIK, "wb") as f:
    pickle.dump(data, f)

## Data import

In [728]:
import pickle
pool_len = 20
PIK = "testQMD_pool"+str(pool_len)+".dat"

with open(PIK, "rb") as f:
    dataold = pickle.load(f)

## Step-by-step checking

In [413]:
print('ModelNames: ' + repr(modeltest.ModelNames))

ModelNames: ['sx_', 'sy_', 'sz_', 'sy_sx_', 'sz_sx_', 'sy_sz_', 'sy_sz_sx_']


In [414]:
ids = 99
print("True model: " + repr(collectTrueNames[ids][0]))

True model: 'sz_sx_'


In [409]:
print("Best model found: " + repr(modeltest.ModelNames[np.argmax(np.array(collectLogL[ids]))]) )

Best model found: 'sy_sz_'


In [353]:
collectTrueParams[ids]

array([[ 0.63612677,  0.41478088]])

In [163]:
evo.getH(collectTrueParams[ids], collectTrueOpList[ids])

array([[ 0.18774291+0.j        ,  0.00000000-0.46414783j],
       [ 0.00000000+0.46414783j, -0.18774291+0.j        ]])

In [203]:
collectLogL[ids]

[-356.34959552749945,
 -309.98335005268302,
 -329.44589238992444,
 -371.01231960058362,
 -266.04046756817871,
 -109.89148827249306,
 -304.02204699679373]

In [165]:
collectKfactorsDict[ids]

{"['sx_']VS['sy_']": 2.466545003709314e+50,
 "['sx_']VS['sy_sx_']": 0.0,
 "['sx_']VS['sy_sz_']": 1.1102230246251565e-16,
 "['sx_']VS['sy_sz_sx_']": 1.2325626075337937e-09,
 "['sx_']VS['sz_']": 1437094397674.96,
 "['sx_']VS['sz_sx_']": 1.0136122796027082e+43,
 "['sy_']VS['sx_']": 0.0,
 "['sy_']VS['sy_sx_']": 0.0,
 "['sy_']VS['sy_sz_']": 0.0,
 "['sy_']VS['sy_sz_sx_']": 0.0,
 "['sy_']VS['sz_']": 0.0,
 "['sy_']VS['sz_sx_']": 4.1094416647347032e-08,
 "['sy_sx_']VS['sx_']": 8.2851831811340211e+37,
 "['sy_sx_']VS['sy_']": 2.0435777180242562e+88,
 "['sy_sx_']VS['sy_sz_']": 9.5849849787552692e+21,
 "['sy_sx_']VS['sy_sz_sx_']": 1.021200708430138e+29,
 "['sy_sx_']VS['sz_']": 1.1906590333318505e+50,
 "['sy_sx_']VS['sz_sx_']": 8.3979634111552724e+80,
 "['sy_sz_']VS['sx_']": 8643918795384441.0,
 "['sy_sz_']VS['sy_']": 2.1320614717224526e+66,
 "['sy_sz_']VS['sy_sx_']": 0.0,
 "['sy_sz_']VS['sy_sz_sx_']": 10654171.192689272,
 "['sy_sz_']VS['sz_']": 1.2422127274804269e+28,
 "['sy_sz_']VS['sz_sx_']": 8.7

### applying directly the champion rule as the highest BayesFactor

In [417]:
performance = []
for ids in range(100):
    performance.append(collectTrueNames[ids][0]==modeltest.ModelNames[np.argmax(np.array(collectLogL[ids]))])
print("Correctly estimated model in " + str(100*sum(performance)/len(performance)) + "% of cases")

Correctly estimated model in 64.0% of cases


### tournament rule among Bayes Factors

In [610]:
prolst=np.arange(len(modeltest.ModsOpList))

prooutlst = np.empty(0)
for i in range(len(prolst)):
    for j in range(len(prolst)):
        if i is not j:
            prooutlst = np.append(prooutlst, np.array([prolst[i],prolst[j]]) )
prooutlst

array([ 0.,  1.,  0.,  2.,  0.,  3.,  0.,  4.,  0.,  5.,  0.,  6.,  1.,
        0.,  1.,  2.,  1.,  3.,  1.,  4.,  1.,  5.,  1.,  6.,  2.,  0.,
        2.,  1.,  2.,  3.,  2.,  4.,  2.,  5.,  2.,  6.,  3.,  0.,  3.,
        1.,  3.,  2.,  3.,  4.,  3.,  5.,  3.,  6.,  4.,  0.,  4.,  1.,
        4.,  2.,  4.,  3.,  4.,  5.,  4.,  6.,  5.,  0.,  5.,  1.,  5.,
        2.,  5.,  3.,  5.,  4.,  5.,  6.,  6.,  0.,  6.,  1.,  6.,  2.,
        6.,  3.,  6.,  4.,  6.,  5.])

In [611]:
modelNames = modeltest.ModelNames
modelNames

['sx_', 'sy_', 'sz_', 'sy_sx_', 'sz_sx_', 'sy_sz_', 'sy_sz_sx_']

In [642]:
threshold = 1

def findWinners(modelNames, proBayesFactorsList):
    proBayesFactorNames=[]
    for i in range(int(len(prooutlst)/2)):
        proBayesFactorNames.append("")
        if proBayesFactorsList[i] > threshold:
            proBayesFactorNames[-1]= str(modelNames[int(prooutlst[2*i])])
        elif proBayesFactorsList[i] < 1/threshold:
            proBayesFactorNames[-1]= str(modelNames[int(prooutlst[2*i+1])])
            
    return(proBayesFactorNames)

In [643]:
def ChampbyTourn(modelNames, collectLogL_single):
    proBayesFactorsList=[]
    for i in range(int(len(prooutlst)/2)):
        proBayesFactorsList.append(bayf.BayesFactorfromLogL(collectLogL_single[int(prooutlst[2*i])], collectLogL_single[int(prooutlst[2*i+1])] ))
    
    proBayesFactorNames = findWinners(modelNames, proBayesFactorsList)
    
    wincount = np.array(list((map(lambda testmodel: proBayesFactorNames.count(testmodel),  modeltest.ModelNames))))
    winner = modeltest.ModelNames[np.argmax(wincount)]
    
    return(winner)

In [644]:
performance = []
for ids in range(100):
    performance.append(collectTrueNames[ids][0]== ChampbyTourn(modelNames, collectLogL[ids]) )
print("Correctly estimated model in " + str(100*sum(performance)/len(performance)) + "% of cases")

Correctly estimated model in 64.0% of cases


#### Breakdown of the comparisons

In [718]:
ids = 25

In [719]:
collectTrueNames[ids][0]

'sy_sz_'

In [720]:
modeltest.ModelNames[np.argmax(np.array(collectLogL[ids]))]

'sy_sx_'

In [721]:
ChampbyTourn(modelNames, collectLogL[ids])

'sy_sx_'

In [722]:
proBayesFactorsList=[]
for i in range(int(len(prooutlst)/2)):
    proBayesFactorsList.append(bayf.BayesFactorfromLogL(collectLogL[ids][int(prooutlst[2*i])], collectLogL[ids][int(prooutlst[2*i+1])] ))

proBayesFactorNames = findWinners(modelNames, proBayesFactorsList)
    
wincount = np.array(list((map(lambda testmodel: proBayesFactorNames.count(testmodel),  modeltest.ModelNames))))

wincount

array([ 6,  0,  4, 12,  2, 10,  8])

In [662]:
modelNames

['sx_', 'sy_', 'sz_', 'sy_sx_', 'sz_sx_', 'sy_sz_', 'sy_sz_sx_']

In [663]:
(collectKfactorsDict[ids])

{"['sx_']VS['sy_']": 0.0,
 "['sx_']VS['sy_sx_']": 6.6871167139478769e+45,
 "['sx_']VS['sy_sz_']": 0.0,
 "['sx_']VS['sy_sz_sx_']": 0.0,
 "['sx_']VS['sz_']": 0.034020909321501924,
 "['sx_']VS['sz_sx_']": 0.0,
 "['sy_']VS['sx_']": 3.6125900373512685e+65,
 "['sy_']VS['sy_sx_']": 2.4157811219413252e+111,
 "['sy_']VS['sy_sz_']": 0.0,
 "['sy_']VS['sy_sz_sx_']": 2.0817747791228504e+39,
 "['sy_']VS['sz_']": 1.2290359807648863e+64,
 "['sy_']VS['sz_sx_']": 0.0,
 "['sy_sx_']VS['sx_']": 0.0,
 "['sy_sx_']VS['sy_']": 0.0,
 "['sy_sx_']VS['sy_sz_']": 0.0,
 "['sy_sx_']VS['sy_sz_sx_']": 0.0,
 "['sy_sx_']VS['sz_']": 0.0,
 "['sy_sx_']VS['sz_sx_']": 0.0,
 "['sy_sz_']VS['sx_']": 1.2745003492793297e+94,
 "['sy_sz_']VS['sy_']": 3.5279407187142895e+28,
 "['sy_sz_']VS['sy_sx_']": 8.5227325875982124e+139,
 "['sy_sz_']VS['sy_sz_sx_']": 7.3443780104600535e+67,
 "['sy_sz_']VS['sz_']": 4.3359660813054559e+92,
 "['sy_sz_']VS['sz_sx_']": 1811744.1069892626,
 "['sy_sz_sx_']VS['sx_']": 1.7353414373064039e+26,
 "['sy_sz_s